# 反向传播

In [2]:
# tf.constant 创建一个张量
# tf.constant(
#     value, dtype=None, shape=None, name='Const'
# )
# value是必选项，dtype、shape、name是可选项
# value可以是一个标量，也可以是python的一个列表
# dtype：数据类型，可以是float32,也可以是float64
# shape: 表示张量的“形状”，即维数以及每一维的大小,
# 1、如果第一个参数value是数字时，张量的所有元素都会用该数字填充
# 例如：tf.constant(-1, shape=[2, 3])
# 结果:  [[-1 -1 -1]
#        [-1 -1 -1]]
# 2、而当第一个参数value是一个列表时，注意列表的长度必须小于等于第三个参数shape的大小（即各维大小的乘积）
# 例如：tensor=tf.constant([1, 2, 3, 4, 5, 6, 7], shape=[2, 3])
# 列表大小为7，而shape大小为2*3=6，无法正确填充，所以发生了错误。
# 3、而如果列表大小小于shape大小，则会用列表的最后一项元素填充剩余的张量元素
# 例如：tensor=tf.constant([1, 2], shape=[1, 4, 3])
# 结果： [[[1 2 2]
#         [2 2 2]
#         [2 2 2]
#         [2 2 2]]]
# 第四个参数name可以是任何内容，主要是字符串就行
# 【注意】v2.2.0没有了第五个参数verify_shape默认为False，如果修改为True的话表示检查value的形状与shape是否相符，如果不符会报错。
# --------------------------------------------------------------------------------------
# tf.Variable(
#     initial_value=None, trainable=None, validate_shape=True, caching_device=None,
#     name=None, variable_def=None, dtype=None, import_scope=None, constraint=None,
#     synchronization=tf.VariableSynchronization.AUTO,
#     aggregation=tf.compat.v1.VariableAggregation.NONE, shape=None
# )
#
# Vatiable是tensorflow的变量节点，通过Variable方法创建，并且需要传递初始值。在使用前需要通过tensorflow的初始化方法进行初始化
#
# W = tf.Variable(
#                 initial_value=tf.zeros([9, 5]),  
#                         # 初始值，必填，张量或可以转换为张量的Python对象。初始值必须有指定一个形状，除非`validate_shape`设置为False。
#                 trainable=True,  
#                         # 如果`True`，则默认值也将变量添加到图形中集合`GraphKeys.TRAINABLE_VARIABLES`。
#                         #这个集合用作“Optimizer”类使用的默认变量列表
#                 collections=None,  
#                         # 图表集合键的列表。新的变量被添加到这些集合。默认为`[GraphKeys.GLOBAL_VARIABLES]`。
#                 validate_shape=True, 
#                         # 如果`False`，允许变量用初始化未知形状的值。如果“True”，默认的形状`initial_value`必须是已知的。
#                  caching_device=None,  
#                         # 可选设备字符串，描述变量的位置应该被缓存以供阅读。默认为变量的设备。如果不是“None”，则缓存在另一个设备上。
#                         #典型的用途是缓存在使用变量 的Ops所在的设备上进行重复数据删除复制`Switch`和其他条件语句。
#                  name='W',  
#                         # 变量的可选名称。默认为“Variable”并获取自动去重（Variable_1,Variable_2....）。
#                  variable_def=None,
#                         # `VariableDef`协议缓冲区。如果不是“无”，则重新创建变量对象及其内容，引用变量的节点在图中，必须已经存在。
#                         #图形没有改变。`variable_def`和其他参数是互斥的。
#                 dtype=tf.float32,
#                         # 如果设置，initial_value将被转换为给定的类型。如果`None'，数据类型将被保存
#                         #（如果`initial_value`是一个张量），或者“convert_to_tensor”来决定。
#                 expected_shape=None,  
#                         # 张量的Shape。如果设置，initial_value需要符合这个形状。
#                  import_scope=None
#                         # 可选的字符串。名称范围添加到`Variable.`仅在从协议缓冲区初始化时使用。
#                     ) 
# tf.Variable() 和tf.get_variable()
# 1、使用tf.Variable时，如果检测到命名冲突，系统会自己处理。使用tf.get_variable()时，系统不会处理冲突，而会报错
# 2、基于这两个函数的特性，当我们需要共享变量的时候，需要使用tf.get_variable()。在其他情况下，这两个的用法是一样的
# 3、由于tf.Variable() 每次都在创建新对象，所有reuse=True 和它并没有什么关系。对于get_variable()，来说，
# 如果已经创建的变量对象，就把那个对象返回，如果没有创建变量对象的话，就创建一个新的。

#-----------------------------------------------------------------------------

In [4]:
import tensorflow as tf

In [5]:
w = tf.Variable(tf.constant(5, dtype=tf.float32))
# lr : learn rate 学习率
lr = 0.1
# 循环次数
epoch = 40

for epoch in range(epoch):  # for epoch 定义顶层循环，表示对数据集循环epoch次，此例数据集数据仅有1个w,初始化时候constant赋值为5，循环40次迭代。
    #tf.GradientTape(
    #    persistent=False, watch_accessed_variables=True
    #)
    # 用于求导
    # persistent=True 表示设置了持久，可以多次调用，默认是False，调用一次后，上下文中就没有了
    # watch_accessed_variables=False 表示禁用自动跟踪被watch的变量，默认是True
    with tf.GradientTape() as tape:  # with结构到grads框起了梯度的计算过程。
        loss = tf.square(w + 1)
    grads = tape.gradient(loss, w)  # .gradient函数告知谁对谁求导
    # print(grads)
    w.assign_sub(lr * grads)  # .assign_sub 对变量做自减 即：w -= lr*grads 即 w = w - lr*grads
    print("After %s epoch,w is %f,loss is %f" % (epoch, w.numpy(), loss))

# lr初始值：0.2   请自改学习率  0.001  0.999 看收敛过程
# 最终目的：找到 loss 最小 即 w = -1 的最优参数w

After 0 epoch,w is 3.800000,loss is 36.000000
After 1 epoch,w is 2.840000,loss is 23.040001
After 2 epoch,w is 2.072000,loss is 14.745600
After 3 epoch,w is 1.457600,loss is 9.437184
After 4 epoch,w is 0.966080,loss is 6.039798
After 5 epoch,w is 0.572864,loss is 3.865470
After 6 epoch,w is 0.258291,loss is 2.473901
After 7 epoch,w is 0.006633,loss is 1.583297
After 8 epoch,w is -0.194694,loss is 1.013310
After 9 epoch,w is -0.355755,loss is 0.648518
After 10 epoch,w is -0.484604,loss is 0.415052
After 11 epoch,w is -0.587683,loss is 0.265633
After 12 epoch,w is -0.670147,loss is 0.170005
After 13 epoch,w is -0.736117,loss is 0.108803
After 14 epoch,w is -0.788894,loss is 0.069634
After 15 epoch,w is -0.831115,loss is 0.044566
After 16 epoch,w is -0.864892,loss is 0.028522
After 17 epoch,w is -0.891914,loss is 0.018254
After 18 epoch,w is -0.913531,loss is 0.011683
After 19 epoch,w is -0.930825,loss is 0.007477
After 20 epoch,w is -0.944660,loss is 0.004785
After 21 epoch,w is -0.95572